In [2]:
import pandas as pd
import numpy as np
import sqlite3
import statsmodels.formula.api as smf
import wrds
from regtabletotext import prettify_result
from dotenv import load_dotenv

from plotnine import *
from mizani.formatters import comma_format, percent_format
from datetime import datetime

In [3]:
from sqlalchemy import create_engine
import os

load_dotenv()

connection_string = (
  "postgresql+psycopg2://"
 f"{os.getenv('WRDS_USER')}:{os.getenv('WRDS_PASSWORD')}"
  "@wrds-pgdata.wharton.upenn.edu:9737/wrds"
)

wrds = create_engine(connection_string, pool_pre_ping=True)

In [4]:
start_date = "2020-01-01"
end_date = "2022-12-31"

In [29]:
ibes_query = (f"""
                        select ticker, pyear, pmon, actual, anndats, 
                        suescore, surpmean, surpstdev, measure, fiscalp, oftic 
                        from ibes.surpsum
                        where anndats between '{start_date}' and '{end_date}' 
                        and USFIRM = 1 and (measure = 'EPS' or measure = 'SAL' or measure = 'NET') and suescore is not null
                        """)
ibes = pd.read_sql_query(
  sql=ibes_query,
  con=wrds,
  parse_dates={"anndats"}
)

In [30]:
ibes

,ticker,pyear,pmon,actual,anndats,suescore,surpmean,surpstdev,measure,fiscalp,oftic
0,000V,2019.0,12.0,-92.400,2020-03-13,-0.33978,-81.18500,33.00618,EPS,ANN,EIGR
1,000V,2019.0,12.0,-20.400,2020-03-13,-0.26602,-12.91000,28.15593,EPS,QTR,EIGR
2,000V,2020.0,3.0,-18.600,2020-05-07,1.23778,-24.08000,4.42728,EPS,QTR,EIGR
3,000V,2020.0,6.0,-18.000,2020-08-06,1.04215,-21.40000,3.26250,EPS,QTR,EIGR
4,000V,2020.0,9.0,-15.600,2020-11-05,1.13809,-17.64000,1.79247,EPS,QTR,EIGR
...,...,...,...,...,...,...,...,...,...,...,...
165896,ZYNX,2021.0,12.0,130.301,2022-02-24,-1.70555,131.25840,0.56135,SAL,ANN,ZYXI
165897,ZYNX,2021.0,12.0,40.366,2022-02-24,-1.70622,41.32440,0.56171,SAL,QTR,ZYXI
165898,ZYNX,2022.0,3.0,31.083,2022-04-28,0.12681,31.01800,0.51257,SAL,QTR,ZYXI
165899,ZYNX,2022.0,6.0,36.759,2022-07-28,0.70373,36.29650,0.65721,SAL,QTR,ZYXI


In [31]:
tidy_finance = sqlite3.connect(database="data/ibes.db.sqlite")
(ibes
  .to_sql(name="ibes", 
          con=tidy_finance, 
          if_exists="replace",
          index=False)
)

165901